## Human PCLS: cellular crosstalk with NicheNetNiklas after Fibrotic Cocktail

In [1]:
suppressPackageStartupMessages(library(nichenetr))
suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(patchwork))
suppressPackageStartupMessages(library(data.table))
suppressPackageStartupMessages(library(Matrix))
suppressPackageStartupMessages(library(igraph))
suppressPackageStartupMessages(library(gridExtra))
suppressPackageStartupMessages(library(viridis))

In [2]:
## set working directory
setwd('/home/niklas/projects/niche_environments_FIBROSIS/HUMAN_exvivo/02_figures/ASK_joint/NicheNet/')

### Path to required input data

In [3]:
dge_dir = '/home/niklas/projects/niche_environments_FIBROSIS/PCLS_human/01_data/ASK_joint/DGE_treatment_vs_CC/'
table_dir = '/home/niklas/projects/niche_environments_FIBROSIS/HUMAN_exvivo/01_data/NicheNet_inputs/'
results_dir = '/home/niklas/projects/niche_environments_FIBROSIS/HUMAN_exvivo/01_data/NicheNet_outputs/ligand_activities_FC_CC/'

### Load NicheNet models and networks

In [4]:
## load NicheNet models and networks ##
ligand_target_matrix <- readRDS('/home/niklas/data/nichenet_models/ligand_target_matrix_HUMAN.rds')
lr_network <- readRDS('/home/niklas/data/nichenet_models/ligand_receptor_network_HUMAN.rds')
weighted_networks_lr <- readRDS('/home/niklas/data/nichenet_models/weighted_ligand_receptor_network_HUMAN.rds')

### Function to perform NicheNet Ligand activity analysis

In [5]:
## function to perform ligand activity analysis ##
ligand_activity_analysis <- function(sender_ct, receiver_ct, geneset_oi, 
                            geneset_title,
                            pct_expr_table, pct_thresh = 0.10,
                            pearson_thresh = 0.08){
    
    
    ## retrieve genes expressed by receiver
    expr_genes_receiver = rownames(pct_expr_table[pct_expr_table[, receiver_ct] > pct_thresh, ])
    background_expr_genes = expr_genes_receiver %>% .[. %in% rownames(ligand_target_matrix)]
    
    ## retrieve genes expressed by sender
    list_expr_genes_sender = lapply(sender_ct, function(x){rownames(pct_expr_table[pct_expr_table[, x] > pct_thresh, ])})
    expr_genes_sender = list_expr_genes_sender %>% unlist() %>% unique()
    
    ## status message ##
    print(paste0("Using ", length(geneset_oi), " genes differently regulated genes in ", receiver_ct, " (",
                 geneset_title, ")"))
    
    ### STEP1: Ligand activity analysis ###
    ## Define a set of potential ligands and receptors 
    # retrieve ligands and receptors
    ligands = lr_network %>% pull(from) %>% unique()
    receptors = lr_network %>% pull(to) %>% unique()
    # ligands expressed by sender celltypes
    expr_ligands = intersect(ligands, expr_genes_sender) 
    # receptor expressed by receiver celltypes
    expr_receptors = intersect(receptors, expr_genes_receiver)
    ### status messages ###
    print(paste0("Expressed Ligands ", length(expr_ligands)))
    print(paste0("Expressed Receptors ", length(expr_receptors)))
    
    ## filter ligands
    # only consider ligands with matching receptors (according to NicheNets databases)
    potential_ligands = lr_network %>% filter(from %in% expr_ligands & to %in% expr_receptors) %>%
                        pull(from) %>% unique()
    ### status message ###
    print(paste0("Potential Ligands ", length(potential_ligands)))
    
    ## predict ligand activities
    ligand_activities = predict_ligand_activities(geneset = geneset_oi,
                                                  background_expressed_genes = background_expr_genes,
                                                  ligand_target_matrix = ligand_target_matrix,
                                                  potential_ligands = potential_ligands)

    ## rank ligands by pearson correlation coefficient
    ligand_activities = ligand_activities %>% arrange(-pearson) %>% mutate(rank = rank(desc(pearson)))
    
    # filter consider ligands with pearson's correlation >= pearson_tresh
    ligand_activities = ligand_activities %>% filter(pearson >= pearson_thresh)
    
    ### status message ###
    print(paste0("Top ranked ligands ", length(ligand_activities$test_ligand)))
    
    return(ligand_activities)  
}

In [6]:
target_gene_prediction <- function(best_upstream_ligands,geneset_oi, target_thresh = 0.33, n_targets = 500){
    
    ## identify ligand targets
    active_ligand_target_links_df = best_upstream_ligands %>% 
                                    lapply(get_weighted_ligand_target_links,geneset = geneset_oi, ligand_target_matrix = ligand_target_matrix, n = n_targets) %>% bind_rows() %>% drop_na()
    active_ligand_target_links = prepare_ligand_target_visualization(ligand_target_df = active_ligand_target_links_df, ligand_target_matrix = ligand_target_matrix, cutoff = target_thresh)
    
    ## reformat data
    order_ligands = intersect(best_upstream_ligands, colnames(active_ligand_target_links)) %>% rev() %>% make.names()
    order_targets = active_ligand_target_links_df$target %>% unique() %>% 
                    intersect(rownames(active_ligand_target_links)) %>% make.names()
    rownames(active_ligand_target_links) = rownames(active_ligand_target_links) %>% make.names() 
    colnames(active_ligand_target_links) = colnames(active_ligand_target_links) %>% make.names() # make.names() for heatmap visualization of genes like H2-T23
    
    ## final ligand-target heatmap
    vis_ligand_target = active_ligand_target_links[order_targets,order_ligands] %>% t()
    
    ## output: ligand-target matrix
    return(vis_ligand_target) 
    
}

### Run NicheNet iteratively to detect crosstalk between all cells for FC vs CC

In [7]:
## read avg expression table
avg_expr <- read.csv(paste0(table_dir, '220207_ASK_joint_Human_PCLS_CC_FC_avg_expr_SCALED.csv'), row.names = 1, check.names = F, header = T)
pct_expr <- read.csv(paste0(table_dir, '220207_ASK_joint_Human_PCLS_CC_FC_pct_expr.csv'), row.names = 1, check.names = F, header = T)

In [8]:
cell_type_names <- as.vector(colnames(avg_expr))
cell_type_labels <- c('Airway_Epithelium','Alveolar_Epithelium',
                      'capillary_EC', 'vascular_EC', 'lymphatic_EC',
                      'Fibroblasts','SMC_Pericytes',
                      'Macrophages','DC','Mast_cells',
                      'B_cells','Plasma_cells','T_cells','NK_cells')

In [9]:
## create final results matrix
quant_results_df <- data.frame(matrix(ncol = length(cell_type_names), nrow = length(cell_type_names)))
colnames(quant_results_df) <- cell_type_names # RECEIVER (!!!)
rownames(quant_results_df) <- cell_type_names # SENDER (!!!)
quant_results_df

,Airway Epithelium,Alveolar Epithelium,capillary EC,vascular EC,lymphatic EC,Fibroblasts,SMC/Pericytes,Macrophages,DC,Mast cells,B cells,Plasma cells,T cells,NK cells
,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
Airway Epithelium,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Alveolar Epithelium,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
capillary EC,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
vascular EC,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
lymphatic EC,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Fibroblasts,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
SMC/Pericytes,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Macrophages,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
DC,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [10]:
qual_results_df <- copy(quant_results_df)
qual_results_df

,Airway Epithelium,Alveolar Epithelium,capillary EC,vascular EC,lymphatic EC,Fibroblasts,SMC/Pericytes,Macrophages,DC,Mast cells,B cells,Plasma cells,T cells,NK cells
,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
Airway Epithelium,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Alveolar Epithelium,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
capillary EC,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
vascular EC,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
lymphatic EC,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Fibroblasts,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
SMC/Pericytes,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Macrophages,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
DC,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


### Run NicheNet

In [11]:
for(i in 1:length(cell_type_names)){
    
    ## step 1: define RECEIVER and SENDER cells
    all_senders <- cell_type_names
    all_sender_labels <- cell_type_labels
    receiver_ct <- cell_type_names[i]
    receiver_labels <- cell_type_labels[i]
    
    ## step 2: define geneset of interest
    # read dge table
    dge_table <- read.csv(paste0(dge_dir, '211206_PCLS_human_ASK_joint_' , cell_type_labels[i], '_FC_vs_CC_DGE_results.csv'))
    # only significant genes: pval_adj < 0.05
    dge_table = dge_table %>% filter(qval < 0.05)
    # only genes expressed in at least 10% per group
    dge_table = dge_table %>% filter(pct.CCs > 0.1)
    dge_table = dge_table %>% filter(pct.FCs > 0.1)
    # only upregulated genes with log2fc > 0.25
    dge_genes_up = dge_table %>% filter(log2fc > 0.25)
    geneset_oi <- dge_genes_up$gene
    geneset_title <- paste0(cell_type_names[i], ' (FC vs. CC)')
    print(paste0(cell_type_names[i], ' (FC vs. CC): Proceeding with geneset of interest of length: ', length(geneset_oi)))
    
    ## step 3: run ligand activity analysis
    top_ligands_table <- ligand_activity_analysis(all_senders, receiver_ct, geneset_oi = geneset_oi,
                                           geneset_title = geneset_title, pct_thresh = 0.10,
                                           pct_expr_table = pct_expr, pearson_thresh = 0.05)
    
    ## dissect crosstalk between all pairs of cell types quantitatively and qualitatively
    
    # for each sender cell type check which top ligands are upregulated
    for(i in 1:length(all_senders)){
        sender_ct <- all_senders[i]
        sender_label <- all_sender_labels[i]
        # read respective DGE table
        # read dge table
        dge_table <- read.csv(paste0(dge_dir, '211206_PCLS_human_ASK_joint_' , sender_label, '_FC_vs_CC_DGE_results.csv'))
        
        # subset DGE table to top ligands
        ligands_dge = dge_table %>% filter(gene %in% top_ligands_table$test_ligand) 
        
        # filter DGE table by logFC and q-Value
        # if logFC > 0.25 and q-Value < 0.05, a ligand is upregulated
        ligands_dge = ligands_dge %>% filter(qval < 0.05)
        ligands_dge = ligands_dge %>% filter(log2fc > 0.25)
        
        ## add info whether gene is upregulated in SENDER cell type to ligand activity table
        top_ligands_table <- top_ligands_table %>% mutate(new_col = case_when(test_ligand %in% ligands_dge$gene ~ 1,
                                         !(test_ligand %in% ligands_dge$gene) ~ 0))
        colnames(top_ligands_table)[colnames(top_ligands_table) == 'new_col'] <- paste0(sender_label, '_UP')
        
        ### quantitative approach ### 
        # count genes in resulting DGE table --> no. of upregulated top ligands
        ct_quant_score <- length(ligands_dge$gene)
        cat('\n')
        print(paste0(sender_ct, ' --> ', receiver_ct, ' - Top ranked ligands: ', length(top_ligands_table$test_ligand)))
        print(paste0(sender_ct, ' --> ', receiver_ct, ' - Thereof upregulated: ', ct_quant_score))
        cat('\n')
        # fill final results matrix
        quant_results_df[sender_ct, receiver_ct] <- ct_quant_score
        
        ### qualitative approach ### 
        
        # filter ligands results table: upregulated ligands only
        ligands_up = top_ligands_table %>% filter(test_ligand %in% ligands_dge$gene)
        
        # filter avg expression table: only sender cell type
        ligands_avg_expr = avg_expr %>% select(sender_ct)
        # filter avg expression table: only top ligands
        ligands_avg_expr = ligands_avg_expr %>% filter(rownames(ligands_avg_expr) %in% ligands_dge$gene)
        ligands_avg_expr$gene <- rownames(ligands_avg_expr)
        rownames(ligands_avg_expr) <- NULL
        
        # merge ligands results table with ligands avg expression table
        ligands_up <- merge.data.frame(x = ligands_up, y = ligands_avg_expr, by.x = 'test_ligand', by.y = 'gene')
        
        # add new colum to ligand results table: avg expression * ligand score (pearsons rho)
        ligands_up$contribution <- ligands_up[, sender_ct] * ligands_up$pearson
        # sender celltype contribution score equals the sum of the new column
        ct_qual_score <- sum(ligands_up$contribution)
        
        # fill final results matrix
        qual_results_df[sender_ct, receiver_ct] <- ct_qual_score

    }
    
    ## save ligand activity analysis
    print(paste0('Writing ligand activity table to:  ', results_dir, 'nichenet_ligand_act_', receiver_labels, '.csv'))
    write.csv(top_ligands_table, paste0(results_dir, 'nichenet_ligand_act_', receiver_labels, '.csv'))
    
    ## step 4: target gene prediction
    ligand_target_prediction <- target_gene_prediction(best_upstream_ligands = top_ligands_table$test_ligand, geneset_oi = geneset_oi)
    print(paste0('Writing ligand target matrix table to:  ', results_dir, 'nichenet_ligand_target_matrix_', receiver_labels, '.csv'))
    write.csv(ligand_target_prediction, paste0(results_dir, 'nichenet_ligand_target_matrix_', receiver_labels, '.csv'))
}

[1] "Airway Epithelium (FC vs. CC): Proceeding with geneset of interest of length: 469"
[1] "Using 469 genes differently regulated genes in Airway Epithelium (Airway Epithelium (FC vs. CC))"
[1] "Expressed Ligands 308"
[1] "Expressed Receptors 152"
[1] "Potential Ligands 203"
[1] "Top ranked ligands 0"

[1] "Airway Epithelium --> Airway Epithelium - Top ranked ligands: 0"
[1] "Airway Epithelium --> Airway Epithelium - Thereof upregulated: 0"



Note: Using an external vector in selections is ambiguous.
ℹ Use `all_of(sender_ct)` instead of `sender_ct` to silence this message.
ℹ See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.




[1] "Alveolar Epithelium --> Airway Epithelium - Top ranked ligands: 0"
[1] "Alveolar Epithelium --> Airway Epithelium - Thereof upregulated: 0"


[1] "capillary EC --> Airway Epithelium - Top ranked ligands: 0"
[1] "capillary EC --> Airway Epithelium - Thereof upregulated: 0"


[1] "vascular EC --> Airway Epithelium - Top ranked ligands: 0"
[1] "vascular EC --> Airway Epithelium - Thereof upregulated: 0"


[1] "lymphatic EC --> Airway Epithelium - Top ranked ligands: 0"
[1] "lymphatic EC --> Airway Epithelium - Thereof upregulated: 0"


[1] "Fibroblasts --> Airway Epithelium - Top ranked ligands: 0"
[1] "Fibroblasts --> Airway Epithelium - Thereof upregulated: 0"


[1] "SMC/Pericytes --> Airway Epithelium - Top ranked ligands: 0"
[1] "SMC/Pericytes --> Airway Epithelium - Thereof upregulated: 0"


[1] "Macrophages --> Airway Epithelium - Top ranked ligands: 0"
[1] "Macrophages --> Airway Epithelium - Thereof upregulated: 0"


[1] "DC --> Airway Epithelium - Top ranked ligands: 0"
[1]

Warning message:
“Unknown or uninitialised column: `weight`.”
Warning message:
“Unknown or uninitialised column: `target`.”
Warning message:
“Unknown or uninitialised column: `ligand`.”
Warning message:
“Unknown or uninitialised column: `target`.”
Warning message:
“Unknown or uninitialised column: `ligand`.”
Warning message:
“Unknown or uninitialised column: `target`.”
Warning message:
“Unknown or uninitialised column: `ligand`.”
Warning message:
“Unknown or uninitialised column: `target`.”


[1] "Writing ligand target matrix table to:  /home/niklas/projects/niche_environments_FIBROSIS/HUMAN_exvivo/01_data/NicheNet_outputs/ligand_activities_FC_CC/nichenet_ligand_target_matrix_Airway_Epithelium.csv"
[1] "Alveolar Epithelium (FC vs. CC): Proceeding with geneset of interest of length: 540"
[1] "Using 540 genes differently regulated genes in Alveolar Epithelium (Alveolar Epithelium (FC vs. CC))"
[1] "Expressed Ligands 308"
[1] "Expressed Receptors 161"
[1] "Potential Ligands 215"
[1] "Top ranked ligands 199"

[1] "Airway Epithelium --> Alveolar Epithelium - Top ranked ligands: 199"
[1] "Airway Epithelium --> Alveolar Epithelium - Thereof upregulated: 27"


[1] "Alveolar Epithelium --> Alveolar Epithelium - Top ranked ligands: 199"
[1] "Alveolar Epithelium --> Alveolar Epithelium - Thereof upregulated: 39"


[1] "capillary EC --> Alveolar Epithelium - Top ranked ligands: 199"
[1] "capillary EC --> Alveolar Epithelium - Thereof upregulated: 31"


[1] "vascular EC --> Alveolar Epi

Warning message:
“Unknown or uninitialised column: `weight`.”
Warning message:
“Unknown or uninitialised column: `target`.”
Warning message:
“Unknown or uninitialised column: `ligand`.”
Warning message:
“Unknown or uninitialised column: `target`.”
Warning message:
“Unknown or uninitialised column: `ligand`.”
Warning message:
“Unknown or uninitialised column: `target`.”
Warning message:
“Unknown or uninitialised column: `ligand`.”
Warning message:
“Unknown or uninitialised column: `target`.”


[1] "Writing ligand target matrix table to:  /home/niklas/projects/niche_environments_FIBROSIS/HUMAN_exvivo/01_data/NicheNet_outputs/ligand_activities_FC_CC/nichenet_ligand_target_matrix_capillary_EC.csv"
[1] "vascular EC (FC vs. CC): Proceeding with geneset of interest of length: 932"
[1] "Using 932 genes differently regulated genes in vascular EC (vascular EC (FC vs. CC))"
[1] "Expressed Ligands 308"
[1] "Expressed Receptors 164"
[1] "Potential Ligands 205"
[1] "Top ranked ligands 199"

[1] "Airway Epithelium --> vascular EC - Top ranked ligands: 199"
[1] "Airway Epithelium --> vascular EC - Thereof upregulated: 31"


[1] "Alveolar Epithelium --> vascular EC - Top ranked ligands: 199"
[1] "Alveolar Epithelium --> vascular EC - Thereof upregulated: 40"


[1] "capillary EC --> vascular EC - Top ranked ligands: 199"
[1] "capillary EC --> vascular EC - Thereof upregulated: 28"


[1] "vascular EC --> vascular EC - Top ranked ligands: 199"
[1] "vascular EC --> vascular EC - Thereof upregul

In [12]:
write.csv(quant_results_df, paste0(results_dir, '220207_quant_crosstalk_FC_CC.csv'))
quant_results_df

,Airway Epithelium,Alveolar Epithelium,capillary EC,vascular EC,lymphatic EC,Fibroblasts,SMC/Pericytes,Macrophages,DC,Mast cells,B cells,Plasma cells,T cells,NK cells
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
Airway Epithelium,0,27,0,31,32,29,31,32,17,2,0,0,0,10
Alveolar Epithelium,0,39,0,40,40,45,45,40,15,4,1,0,1,8
capillary EC,0,31,0,28,27,33,33,27,14,3,1,1,0,10
vascular EC,0,56,0,53,52,58,61,56,21,7,1,1,1,18
lymphatic EC,0,17,0,14,15,17,17,20,6,3,0,1,0,4
Fibroblasts,0,47,0,41,40,46,48,42,20,5,0,0,1,14
SMC/Pericytes,0,45,0,42,40,46,49,46,16,7,0,0,1,11
Macrophages,0,37,0,32,35,37,38,35,20,5,0,0,0,15
DC,0,3,0,5,5,4,4,4,2,1,0,0,0,2


In [13]:
write.csv(qual_results_df, paste0(results_dir, '220207_qual_crosstalk_FC_CC.csv'))
qual_results_df

,Airway Epithelium,Alveolar Epithelium,capillary EC,vascular EC,lymphatic EC,Fibroblasts,SMC/Pericytes,Macrophages,DC,Mast cells,B cells,Plasma cells,T cells,NK cells
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Airway Epithelium,0,1.0940329,0,1.54659939,1.57506906,1.75838954,1.55431504,1.45757675,0.55545661,0.05736601,0.000000000,0.00000000,0.00000000,0.379090823
Alveolar Epithelium,0,1.3927742,0,1.78199139,1.79875321,2.22525948,2.15323493,1.59685289,0.35091449,0.11444368,0.020813147,0.00000000,0.01754660,0.308706655
capillary EC,0,0.6062182,0,0.57687320,0.54492704,0.73866150,0.73994891,0.63429980,0.17585428,0.09209890,0.043404059,0.05923539,0.00000000,0.148032441
vascular EC,0,2.2383282,0,2.56103671,2.47762538,2.99987014,2.96583388,2.68894870,0.48151332,0.31784705,0.092224931,0.15456168,0.06120471,0.685711307
lymphatic EC,0,0.7822199,0,0.67924936,0.97863685,0.99081639,0.92651700,1.19125584,0.21323504,0.09890368,0.000000000,0.07524307,0.00000000,0.178940469
Fibroblasts,0,1.9839980,0,1.94843079,1.86112678,2.61101810,2.48016030,1.94427959,0.61704524,0.15883907,0.000000000,0.00000000,0.01001364,0.306704695
SMC/Pericytes,0,2.0398041,0,2.55286356,2.14715956,2.68736028,2.74523757,2.46839224,0.44751311,0.14518190,0.000000000,0.00000000,0.01394310,0.348199149
Macrophages,0,2.1302126,0,1.81219163,2.27793164,2.93430422,2.71992210,2.45086681,0.82742580,0.18513000,0.000000000,0.00000000,0.00000000,0.589845572
DC,0,0.1720985,0,0.35298087,0.35334218,0.25707568,0.23542371,0.24609477,0.15437238,0.02224709,0.000000000,0.00000000,0.00000000,0.099146023
